In [0]:
ecom_oct_df = spark.read.options(header='True', inferSchema='True').csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Oct.csv")

ecom_oct_df.limit(5).display()

In [0]:
ecom_oct_df.printSchema()

In [0]:
category_df = ecom_oct_df.select("category_code","category_id","product_id")
category_df.limit(5).display()

In [0]:
ecom_oct_df.groupby("product_id").count().orderBy("count",ascending=False).limit(5).display()

In [0]:
product_df = ecom_oct_df.select('product_id', "brand","price","category_id","category_code").dropDuplicates(["product_id"])
product_df.limit(5).display()

In [0]:
user_df = ecom_oct_df.select('user_id')
user_df.limit(5).display()

In [0]:
views_df = ecom_oct_df.filter(ecom_oct_df.event_type == "view")
views_df.limit(5).display()
carts_df = ecom_oct_df.filter(ecom_oct_df.event_type == "cart")
carts_df.limit(5).display()
purchase_df = ecom_oct_df.filter(ecom_oct_df.event_type == "purchase")
purchase_df.limit(5).display()

In [0]:
views_cart_in_df = views_df.join(carts_df, "product_id","inner")

views_cart_df.limit(5).display()

In [0]:
views_df.join(purchase_df, ["product_id","user_id"],"left").limit(5).display()

In [0]:
carts_df.join(purchase_df, ["product_id","user_id"],"right").limit(5).display()

In [0]:
purchase_df.join(carts_df, ["product_id","user_id"],"right").limit(5).display()

In [0]:
full_df= ecom_oct_df.join(purchase_df, ["product_id","user_id"],"full")
full_df.limit(5).display()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
window = Window.partitionBy("user_id").orderBy(F.desc("event_time"))
rank_df = ecom_oct_df.withColumn("rank", F.row_number().over(window))

In [0]:
rank_df.limit(5).display()


In [0]:
run_sum_df = ecom_oct_df.withColumn("run_sum", F.count('*').over(window))
run_sum_df.filter(run_sum_df.user_id == '512365129').display()

In [0]:
agg_df = ecom_oct_df.groupBy("category_code").agg(
    F.sum(F.when(F.col("event_type") == "view", 1).otherwise(0)).alias("view"),
    F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchase")
)
agg_df = agg_df.withColumn("Conversion Rate", F.round(F.col("purchase")/F.col("view"), 2))
agg_df.orderBy(F.desc("Conversion Rate")).display()